# QA with TikToken code

In [ ]:
import os
import openai
from IPython.display import display, HTML, Markdown
from pprint import pprint

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.callbacks import OpenAICallbackHandler

totals_cb = OpenAICallbackHandler()

print(totals_cb)

In [ ]:
!rm -rf ./tiktoken
!git clone https://github.com/openai/tiktoken

In [ ]:
from langchain.document_loaders import TextLoader

root_dir = 'tiktoken'
chunks = []

# Go through each folder
for dirpath, dirnames, filenames in os.walk(root_dir):
    
    # Go through each file
    for file in filenames:
        try: 
            # Load the file and split
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            chunks += loader.load_and_split()
        except Exception as e: 
            pass


print (f"There are {len(chunks)} chunks\n")
print (chunks[0].page_content[:200])

In [ ]:
!rm -rf ./chroma_db/tiktoken

In [ ]:
# Use this cell only to create a Vector DB from scratch
# Skip it if persisted DB already exists

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")

db = Chroma.from_documents(
    chunks, 
    embedding=embeddings,
    persist_directory="./chroma_db/tiktoken"
) # takes few minutes
db.persist()

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")

db = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma_db/tiktoken"
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

In [ ]:
question = "Give me example code to count tokens in a text"

result = qa_chain({"query": question}, callbacks=[totals_cb])

display(Markdown(result["result"]))

In [ ]:
import tiktoken

# Load the encoding for the desired model
enc = tiktoken.encoding_for_model("gpt-4")

def count_tokens(text):
    # Encode the text into tokens
    tokens = enc.encode(text)

    # Count the number of tokens
    num_tokens = len(tokens)

    return num_tokens

# Example usage
text = "Hello, world! This is an example text."
num_tokens = count_tokens(text)
print("Number of tokens:", num_tokens)

In [ ]:
pprint(result["source_documents"])

In [ ]:
question = "How to get encoding for specific model?"

result = qa_chain({"query": question}, callbacks=[totals_cb])

display(Markdown(result["result"]))

In [ ]:
question = "What encoding_for_model does?"

result = qa_chain({"query": question}, callbacks=[totals_cb])

display(Markdown(result["result"]))

In [ ]:
print(totals_cb)